# Send gRPC and API Calls via Python Scripts to OpenVINO Model Server in OpenShift 

We will show you how to deploy an OpenVINO Model Server (OVMS) service in an OpenShift cluster. Then, we will send gRPC and REST API calls to the AI inference service by invoking Python scripts.

Requirements:
- OpenShift cluster with the API access to a project
- installed [OpenVINO Model Server Operator](https://catalog.redhat.com/software/operators/search?q=openvino)
- JupyterLab environment with Python3 deployed in the cluster

If you don't have an OpenShift account, you can sign up for 30 or 60 day [free trial of Red Hat OpenShift](https://www.openshift.com/try).

## Login to OpenShift with API Token

First, let's login to OpenShift cluster using `oc` tool. 

In the Red Hat OpenShift console, click on your username and select `Copy login command`.

![copy-login.png](notebook-files/copy-login.png)

Click on `Display Token` and your API token will appear.

![log-in-with-token.png](notebook-files/log-in-with-token.png)

Copy `Log in with token` command and paste it in the cell below. The command has your `<user-API-token>` and `<cluster-DNS-name>`.

In [ ]:
!oc login --token=<user-API-token> --server=https://api.<cluster-DNS-name>:6443

Create `ovms-operator` project and go to this project.

In [ ]:
!oc new-project ovms-operator 
!oc project ovms-operator

## Deploy an OVMS Service


Here's the yaml used to configure the OVMS service. We specified `name` to be `ovms-resnet` and `model_path` to be `gs://ovms-public-eu/resnet50-binary`. Also, we defined `model_name` here; we will use this value for gRPC and REST API calls.

In [ ]:
!cat ovms.yaml

Run the cell below to create new OVMS service called `ovms-resnet`.

In [ ]:
!oc apply -f ovms.yaml

Let's see if pod and service were created. They should start with `ovms-resnet`.

In [ ]:
!oc get pod
!oc get service

Check if the service is up and running by making a REST API call.

In [ ]:
!curl http://ovms-resnet.ovms-operator.svc.cluster.local:8081/v1/models/resnet

The `state` parameter should be `AVAILABLE`. Note `version` value, we will use it for gRPC and REST API calls. 

## gRPC API Calls

In this section, we will make gRPC API calls to the OVMS service using sample scripts.

To run sample scripts, we will need to get the serving metadata. `get_serving_meta.py` script has the following arguments:
* **--grpc_address**  
* **--grpc_port**
* **--model_name**
* **--model_version**

Run the cell to get the metadata.

In [ ]:
!python get_serving_meta.py --grpc_address ovms-resnet.ovms-operator.svc.cluster.local \
                            --grpc_port 8080 \
                            --model_name resnet \
                            --model_version 1

For the `jpeg_classificaion.py` script, we specified `--input_name` and `--output_name` arguments, from the metadata output. However, we omitted the `--images_list` argument because we used the default value, `input_images.txt`. 

Let's see what's in that file.

In [ ]:
!cat input_images.txt

As you can see, `input_images.txt` contains paths to images and [class ids](classes.py).

Run a classification inference on list of images, listed in `input_images.txt`.

In [ ]:
!python3 jpeg_classification.py --grpc_address ovms-resnet.ovms-operator.svc.cluster.local \
                                --grpc_port 8080 \
                                --input_name 0 \
                                --output_name 1463 \
                                --images_list input_images.txt

Let's try another script, `grpc_serving_client.py`. It accepts NumPy array as an input for `--images_numpy_path` argument. We will also add `labels_numpy_path`, so we can see if the OVMS service labeled the objects correctly. 

In [ ]:
!python3 grpc_serving_client.py \
        --grpc_address ovms-resnet.ovms-operator.svc.cluster.local \
        --grpc_port 8080 \
        --input_name 0 \
        --output_name 1463 \
        --images_numpy_path imgs.npy \
        --labels_numpy_path lbs.npy \
        --transpose_input False 

## REST API Calls

In this section, we will make REST API calls to the OVMS service using sample scripts. We will use port `8081` as `--rest_port`, as we specified in the `ovms.yaml` when we created the OVMS service.

Run the cell below to get the model status.

In [ ]:
!python rest_get_model_status.py --rest_url http://ovms-resnet.ovms-operator.svc.cluster.local \
                                 --rest_port 8081 \
                                 --model_name resnet

Run the cell below to get serving metadata.

In [ ]:
!python rest_get_serving_meta.py --rest_url http://ovms-resnet.ovms-operator.svc.cluster.local \
                                 --rest_port 8081 \
                                 --model_name resnet

Let's run `rest_serving_client.py` script. It takes `--images_numpy_path` and `--labels_numpy_path` as an input. The script return the same output as `grpc_serving_client.py`.

In [ ]:
!python rest_serving_client.py --rest_url http://ovms-resnet.ovms-operator.svc.cluster.local \
                               --rest_port 8081 \
                               --model_name resnet \
                               --input_name 0 \
                               --output_name 1463 \
                               --images_numpy_path imgs.npy \
                               --labels_numpy_path lbs.npy \
                               --transpose_input False                

## Cleanup

Let's free up resources.

In [ ]:
!oc delete ovms ovms-resnet

## Next Steps

In this notebook, you have learned how to run image classification requests with Python scripts. Next, you can explore other OpenShift OVMS notebooks:

- [Deploy Image Classification with OpenVINO Model Server in OpenShift](../401-model-serving-openshift-resnet/ovms-openshift-resnet.ipynb)
- [Face Detection Multi Model OpenVINO Model Server Deployment in OpenShift](../403-model-serving-openshift-face-detection-dag/ovms-openshift-face-detection-dag.ipynb)